In [9]:
import kaggle
import pandas as pd
import geopandas as gpd

import requests
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'geopandas'

In [4]:
#Kaggle data api
kaggle.api.authenticate()

#download the Kaggle Dataset
kaggle.api.dataset_download_files('dgomonov/new-york-city-airbnb-open-data', path='C:\\Users\\darie\\Desktop\\Python Practice\\Airbnb-Price-Prediction\\data\\new-york-city-airbnb-open-data', unzip=True)

In [ ]:
# get the latest csv file urls for the data sets
# To do this, I go to the website that lists all the downloads ...
airbnb_data_url = 'http://insideairbnb.com/get-the-data.html'
soup = BeautifulSoup(requests.get(airbnb_data_url).text)

# Find the table with the new york city data sets ...
new_york_table = soup.find_all('table',{'class':'new-york-city'})

# Get the first 7 rows in the table (excluding the header row) ...
csv_files = new_york_table[0].find_all('tr')[1:8]

In [6]:
# read the csvs into their own variables depending on the path
# Note: the gz files are zipped, so we need to unzip them when we read them in.
for x in csv_files:
    href = x.find('a')['href']
    print(href)
    
    if 'data/listings.csv.gz' in href:
        airbnb_listings_data = pd.read_csv(href, compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)
        
    elif 'data/calendar.csv.gz' in href:
        airbnb_calendar_data = pd.read_csv(href, compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)
        
    elif 'data/reviews.csv.gz' in href:
        airbnb_reviews_data = pd.read_csv(href, compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)
        
    elif 'visualisations/listings.csv' in href:
        airbnb_listings_visual = pd.read_csv(href)
        
    elif 'visualisations/reviews.csv' in href:
        airbnb_reviews_visual = pd.read_csv(href)
        
    elif 'visualisations/neighbourhoods.csv' in href:
        airbnb_neighbourhoods_visual = pd.read_csv(href)
        
    elif 'visualisations/neighbourhoods.geojson' in href:
        airbnb_neighbourhoods_visual_json = gpd.read_file(href)

http://data.insideairbnb.com/united-states/ny/new-york-city/2020-05-06/data/listings.csv.gz
1
http://data.insideairbnb.com/united-states/ny/new-york-city/2020-05-06/data/calendar.csv.gz
http://data.insideairbnb.com/united-states/ny/new-york-city/2020-05-06/data/reviews.csv.gz
http://data.insideairbnb.com/united-states/ny/new-york-city/2020-05-06/visualisations/listings.csv
http://data.insideairbnb.com/united-states/ny/new-york-city/2020-05-06/visualisations/reviews.csv
http://data.insideairbnb.com/united-states/ny/new-york-city/2020-05-06/visualisations/neighbourhoods.csv
http://data.insideairbnb.com/united-states/ny/new-york-city/2020-05-06/visualisations/neighbourhoods.geojson


In [8]:
print(airbnb_listings_data.head())

print(airbnb_calendar_data.head())

print(airbnb_reviews_data.head())

print(airbnb_listings_visual.head())

print(airbnb_reviews_visual.head())

print(airbnb_neighbourhoods_visual.head())

print(airbnb_neighbourhoods_visual_json.head())

     id                        listing_url       scrape_id last_scraped  \
0  2060  https://www.airbnb.com/rooms/2060  20200506054410   2020-05-09   
1  2595  https://www.airbnb.com/rooms/2595  20200506054410   2020-05-08   
2  3831  https://www.airbnb.com/rooms/3831  20200506054410   2020-05-08   
3  5099  https://www.airbnb.com/rooms/5099  20200506054410   2020-05-09   
4  5121  https://www.airbnb.com/rooms/5121  20200506054410   2020-05-07   

                                        name  \
0                                 Modern NYC   
1                      Skylit Midtown Castle   
2            Cozy Entire Floor of Brownstone   
3  Large Cozy 1 BR Apartment In Midtown East   
4                            BlissArtsSpace!   

                                             summary  \
0                                                NaN   
1  Beautiful, spacious skylit studio in the heart...   
2  Urban retreat: enjoy 500 s.f. floor in 1899 br...   
3  My large 1 bedroom apartment has 